In [1]:
import pandas as pd

df = pd.read_csv('manual_code.csv', encoding='utf-8', sep=';')

In [16]:
df.head(3)

,codmanua,platafor,tipo,modulo,parametr,corpo
0,6675DFA7-A091-4C5E-8630-000023A4AE48,WIN,SQLFUNC,SIM,Extenso_sd,"CREATE procedure dbo.Extenso_sd (@valor float,..."
1,260DC5CE-F4A4-4BB8-A8D3-0001841FBDEB,CS,GWACTIVDEL,SIM,WORKF,"return new GlobalFunctions(user, user.CurrentM..."
2,CCE22EAE-F71D-4418-9722-00028C679034,WIN,ONCANCL,GFP,RECEIFAC,"\r\n\t\t//Só para mensagens de erro, nao bloqu..."


In [17]:
subset = df[(df.platafor == 'WIN') & (df.tipo == 'SQLFUNC')]

In [27]:
df = df[df['platafor'].notnull()]

In [28]:
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

pipe = make_pipeline(
    CountVectorizer(),
    XGBClassifier()
)

pipe.fit(df['corpo'].fillna(''), df['platafor'])

C:\Users\paulo\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:03:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

array(['WIN', 'CS', 'WIN', 'CS', 'MVC'], dtype=object)

In [12]:
df.platafor.value_counts()

MVC        22960
WIN        19979
CS          9854
QWEB        2489
WCF          470
ADMCS        101
WS            87
VUE           15
REP            6
JS             5
WEB            1
JENKINS        1
Name: platafor, dtype: int64

In [98]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(subset['corpo'].values)
arr = X.toarray()

In [128]:
import seaborn as sns

sims = cosine_similarity(arr)

In [106]:
subset['corpo']

0        CREATE procedure dbo.Extenso_sd (@valor float,...
5        CREATE FUNCTION [dbo].[FUN_EXECPOSI_CABIM](@PI...
33       CREATE FUNCTION [dbo].[getDadosEmissaoDecIRSBe...
34       -- CRC_1 - O NIF ou o Código Fonte deve estar ...
36       CREATE FUNCTION dbo.FUN_BACTIV_RESTITUIGR_MOVI...
                               ...                        
55913    CREATE PROCEDURE [dbo].[MAPA_BALANCETEANALITIC...
55926    Create FUNCTION [dbo].[DataMaisRecente] (@dtsi...
55929    --created by [JL] at [2015.01.19]\r\n--last up...
55933    \r\nCREATE VIEW [dbo].[DEMRESULN_PREV_ANO]\r\n...
55944    CREATE VIEW VIEW_PESP_RECEITA\r\nas\r\nselect\...
Name: corpo, Length: 3781, dtype: object

In [113]:
arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [130]:
for i in range(sims.shape[0]):
    for j in range(sims.shape[1]):
        if i == j:
            sims[i][j] = 0

In [153]:
import numpy as np

max_similarity = np.argmax(sims, axis=1)

In [167]:
similarity_values = sims[np.arange(len(sims)), max_similarity]

In [168]:
similarity_values

array([0.91643277, 0.97526535, 1.        , ..., 0.98863065, 0.99969918,
       0.82584298])

In [169]:
subset['most_similar'] = subset.iloc[max_similarity]['corpo'].values
subset['similarity_values'] = similarity_values
subset['most_similar_idx'] = max_similarity

<ipython-input-169-10ababc3dd5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['most_similar'] = subset.iloc[max_similarity]['corpo'].values
<ipython-input-169-10ababc3dd5b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['similarity_values'] = similarity_values
<ipython-input-169-10ababc3dd5b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [178]:
subset.head(5)

,codmanua,platafor,tipo,modulo,parametr,corpo,most_similar,most_similar_idx,similarity_values
0,6675DFA7-A091-4C5E-8630-000023A4AE48,WIN,SQLFUNC,SIM,Extenso_sd,"CREATE procedure dbo.Extenso_sd (@valor float,...","CREATE FUNCTION Extenso (@valor MONEY, @type I...",2578,0.916433
5,878D03C6-DD3F-4D33-A2ED-00065995630B,WIN,SQLFUNC,GFP,FUN_EXECPOSI_CABIM,CREATE FUNCTION [dbo].[FUN_EXECPOSI_CABIM](@PI...,CREATE FUNCTION dbo.FUN_BDACTIV_CABIM(@PI_CLOA...,3739,0.975265
33,91D3F4C0-C1EE-4534-98FC-00242CE284AF,WIN,SQLFUNC,ACS,getDadosEmissaoDecIRSBeneficiario,CREATE FUNCTION [dbo].[getDadosEmissaoDecIRSBe...,CREATE FUNCTION [dbo].[getDadosEmissaoDecIRSBe...,1322,1.000000
34,19A62B7E-F011-49D4-A85F-00244EBDF1E0,WIN,SQLFUNC,BRR,CRC_1,-- CRC_1 - O NIF ou o Código Fonte deve estar ...,CREATE FUNCTION CRC_2(@CODGENER uniqueidentifi...,1639,0.809521
36,80EA66AD-1E45-4EFD-9FA5-002565C6D400,WIN,SQLFUNC,GFP,FUN_BACTIV_RESTITUIGR_MOVINTER,CREATE FUNCTION dbo.FUN_BACTIV_RESTITUIGR_MOVI...,CREATE FUNCTION [dbo].[FUN_BACTIV_RESTITUIGR_E...,3423,0.993248


In [191]:
subset.to_csv('resultados.csv')

In [190]:
subset.iloc[33]

codmanua                          EF10D187-FA69-4380-AA41-01A807BECF4D
platafor                                                           WIN
tipo                                                           SQLFUNC
modulo                                                             GFP
parametr                                           FUN_BDACTIV_DCONGEL
corpo                CREATE FUNCTION dbo.FUN_BDACTIV_DCONGEL(@PI_CL...
most_similar         CREATE FUNCTION dbo.FUN_BDACTIV_ANUL(@PI_CLOAC...
most_similar_idx                                                   124
similarity_values                                             0.988751
Name: 369, dtype: object

In [189]:
print(subset.iloc[33].corpo)

CREATE FUNCTION dbo.FUN_BDACTIV_DCONGEL(@PI_CLOAC VARCHAR(40), @PI_FFINA VARCHAR(40), @PI_TAFEC VARCHAR(10), @PI_DATA1 DATETIME)
RETURNS FLOAT AS
BEGIN
	RETURN
	CASE WHEN @PI_FFINA IS NULL
		THEN
			--(SELECT ISNULL(SUM(dbo.GFPanuac.desconge),0)
			--FROM dbo.GFPanuac
			--WHERE
				--dbo.GFPanuac.codcloac=@PI_CLOAC
				--AND dbo.GFPanuac.data<=@PI_DATA1)
			0
		ELSE
			(SELECT ISNULL(dbo.RoundQG(SUM(CASE WHEN dbo.GFPanuac.valor=0 THEN dbo.GFPlffin.percreal/100*dbo.GFPanuac.desconge ELSE dbo.GFPlffin.valor/dbo.GFPanuac.valor*dbo.GFPanuac.desconge END),2),0)
			FROM dbo.GFPlffin WITH (NOLOCK)
				INNER JOIN dbo.GFPanuac WITH (NOLOCK) ON dbo.GFPlffin.codanuac=dbo.GFPanuac.codanuac
			WHERE
				dbo.GFPanuac.codcloac=@PI_CLOAC
				AND dbo.GFPanuac.data<=@PI_DATA1
				AND dbo.GFPlffin.codffina=@PI_FFINA
				AND dbo.GFPlffin.tipoafec=@PI_TAFEC
				AND dbo.GFPlffin.zzstate = 0
				AND dbo.GFPanuac.zzstate = 0) END
END


In [181]:
print(subset.iloc[2578].corpo)

CREATE FUNCTION Extenso (@valor MONEY, @type INT, @singular VARCHAR(128), @plural VARCHAR(128), @singular_cent VARCHAR(128), @plural_cent VARCHAR(128))
RETURNS VARCHAR(1000)
AS
BEGIN
    DECLARE @v MONEY
    DECLARE @aux VARCHAR(1000), @ext VARCHAR(1000)
    DECLARE @b INT, @c INT, @m INT, @mi INT, @mm INT, @u INT
    DECLARE @p VARCHAR(10), @s VARCHAR(10)
    SET @aux = ''
    SET @ext = ''
    SET @p = ''
    SET @s = ''
    IF @valor < 0
    BEGIN
        SET @valor = -@valor
        SET @ext = 'menos '
    END
    SET @v = FLOOR(@valor / 1000000000000.0)
    SET @b = CAST(@v AS INT)
    SET @valor = @valor - @v * 1000000000000
    SET @v = FLOOR(@valor / 1000000000.0)
    SET @mm = CAST(@v AS INT)
    SET @valor = @valor - @v * 1000000000
    SET @v = FLOOR(@valor / 1000000.0)
    SET @mi = CAST(@v AS INT)
    SET @valor = @valor - @v * 1000000
    SET @v = FLOOR(@valor / 1000.0)
    SET @m = CAST(@v AS INT)
    SET @valor = @valor - @v * 1000
    SET @v = FLOOR(@valor)
    SET @u 